# 0. Import dependencies

In [6]:
import numpy as np
import tensorflow as tf
import re
import time

In [7]:
!python -V #should work on 3.5.4

Python 3.5.4


In [8]:
!pip freeze

argon2-cffi==21.1.0
attrs==22.1.0
backcall==0.2.0
bleach==3.3.1
cffi==1.14.6
colorama==0.4.5
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.3
importlib-metadata==2.1.3
ipykernel==5.5.6
ipython==7.9.0
ipython-genutils==0.2.0
jedi==0.17.2
Jinja2==2.11.3
jsonschema==3.2.0
jupyter-client==6.1.12
jupyter-core==4.6.3
MarkupSafe==1.1.1
mistune==0.8.4
nbconvert==5.6.1
nbformat==5.1.3
notebook==6.2.0
numpy==1.18.5
packaging==20.9
pandocfilters==1.5.0
parso==0.7.1
pickleshare==0.7.5
prometheus-client==0.12.0
prompt-toolkit==2.0.10
protobuf==3.19.6
pycparser==2.21
Pygments==2.11.2
pyparsing==2.4.7
pyrsistent==0.17.3
python-dateutil==2.8.2
pywin32==301
pywinpty==0.5.7
pyzmq==20.0.0
Send2Trash==1.8.0
six==1.16.0
tensorflow==1.0.0
terminado==0.8.3
testpath==0.6.0
tornado==6.1
traitlets==4.3.3
wcwidth==0.2.6
webencodings==0.5.1
win-unicode-console==0.5
zipp==1.2.0


DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.


# 1. Data Preprocessing

### Importing the dataset

In [9]:
lines = open("movie_lines.txt",encoding = "UTF-8",errors="ignore").read().split("\n")
conversations = open("movie_conversations.txt",encoding = "UTF-8",errors="ignore").read().split("\n")

In [10]:
lines

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?',
 'L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".',
 'L867 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ What good stuff?',
 "L866 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ I figured yo

In [11]:
conversations

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

### Creating a dictionary that maps ids to lines

In [12]:
id2line = {}
for line in lines:
    _line = line.split(" +++$+++ ")
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [13]:
id2line

{'L39209': 'Holy mother of God...',
 'L361648': "And do good things with her...share the thing...all that bullshit is true, y'know...find someone and hold on all that...Where's Linda?",
 'L296207': "I don't know -- I don't know--",
 'L350631': 'Peppermint... alright.',
 'L279185': 'Not the point.',
 'L168874': 'Well... maybe someone is, like... giving you these dream steroids. And sleepwalking ...is someone showing you the way.',
 'L244210': 'What do I...?',
 'L290393': 'When you were outside with Bill.',
 'L562528': 'Not yet. Not about this, either.',
 'L469193': "Don't do this",
 'L374020': 'Well, maybe they do that in Riverdale, but down here we take cash.  Now, you got it or not?',
 'L50075': 'What does this matter to you?',
 'L355942': "I've got men searching the building and the grounds now.",
 'L518513': 'Gibarian.',
 'L438879': "Yeah. You're so lucky. He really loves you, too. He's always telling Walter how wonderful you are.",
 'L4697': "Being a hot dog's worked pretty well fo

### Creating a list of all the conversations

In [14]:
conversations_id=[]
for conversation in conversations[:-1]:
    _conversation = conversation.split(" +++$+++ ")[-1]
    _conversation = _conversation.replace("]","")    
    _conversation = _conversation.replace("[","")    
    _conversation = _conversation.replace("'","")
    _conversation = _conversation.replace(" ","")
    _conversation = _conversation.split(",")
    conversations_id.append(_conversation)

In [15]:
conversations_id

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

### Creating separately the questions and answers

In [16]:
questions = []
answers = []
for conversation in conversations_id:
    for i in range(0,len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [17]:
questions

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...',
 "C'esc ma tete. This is my head",
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have n

In [18]:
answers

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.',
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 "That's because it's such a nice one.",
 'Forget French.',
 "Well, there's someone I think might be --",
 'Where?',
 "I 

### Cleaning the texts

In [19]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"i've","i have",text)
    text = re.sub(r"he's","he is",text)    
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)    
    text = re.sub(r"what's","what is",text)  
    text = re.sub(r"\'ll"," will",text)
    text = re.sub(r"\'ve"," have",text)
    text = re.sub(r"\'re"," are",text)
    text = re.sub(r"\'d"," would",text)
    text = re.sub(r"\'ll"," will",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"didn't","do not",text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]","",text)
    return text

### Cleaning the questions and answers

In [20]:
clean_questions = []
for question in questions:
    _question = clean_text(question)
    clean_questions.append(_question)

In [21]:
clean_answers = []
for answer in answers:
    _answer = clean_text(answer)
    clean_answers.append(_answer)

In [22]:
clean_questions

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 'no no it is my fault  we do not have a proper introduction ',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i can not date until she does',
 'why',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head",
 'right  see  you are ready for the quiz',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point o

In [23]:
clean_answers

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 "okay then how 'bout we try out some french cuisine  saturday  night",
 'forget it',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i can not date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do',
 'right  see  you are ready for the quiz',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone",
 'that is because it is such a nice one',
 'forget french',
 "well there's someone i think might be ",
 'where',
 "i counted on you to help my caus

### Creating the frequency dictionary of words in the corpus

In [24]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word]=0
        word2count[word]+=1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word]=0
        word2count[word]+=1

In [25]:
word2count

{'nihilistic': 2,
 'screech': 3,
 'freebasemaybe': 1,
 'archives': 7,
 'outta': 556,
 'beenit': 1,
 'guideline': 1,
 "ricky's": 5,
 'sheeett!!!': 2,
 'motorcyclethe': 2,
 'reconnaissance': 6,
 'reek': 8,
 'mymy': 17,
 'peopled': 2,
 'burg': 10,
 'iraqis': 36,
 'catching': 65,
 'uhthe': 1,
 'rubinstein': 1,
 'stubbins': 10,
 'burnham': 7,
 'bertthe': 1,
 'paratrooped': 1,
 'stutter!': 1,
 'pianos': 9,
 'york': 690,
 "bartlett's": 4,
 'braniff': 4,
 'santino': 9,
 'adieu': 2,
 'xi': 2,
 'mortified': 2,
 'venetian': 1,
 "polygamy's": 2,
 'transshipment': 3,
 "more'": 2,
 'run!': 36,
 'showy!': 2,
 'blockhead!': 1,
 'shazam': 1,
 'russki': 3,
 'gator': 4,
 'austrians': 3,
 'assets!': 2,
 'sheila!': 3,
 'orientation': 10,
 "take'm": 1,
 'dullards': 2,
 "sloviak's": 2,
 'grinds': 1,
 "wouldon't": 16,
 'backyard!': 2,
 'linkski': 2,
 'tork': 1,
 "w'sup": 1,
 "ind'n": 1,
 'twentyfourhour': 1,
 'gladiators': 4,
 'sircan': 2,
 'wellof': 1,
 'stretch!': 1,
 'publicly': 15,
 'feminine': 14,
 'stew

### Choosing a threshold and filtering it

In [26]:
threshold = 20

In [27]:
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if(count >= threshold):
        questionswords2int[word] = word_number
        word_number += 1
        
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if(count >= threshold):
        answerswords2int[word] = word_number
        word_number += 1

### Adding last few tokens to dictionaries

In [28]:
tokens = ["<PAD>","<EOS>","<OUT>","<SOS>"]
for token in tokens:
    questionswords2int[token] = len(questionswords2int)+1
for token in tokens:
    answerswords2int[token] = len(answerswords2int)+1

### Reverse mapping the dictionaries

In [29]:
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}

### Adding EOS at the end and translating QnA to integers and replacing filtered words with <OUT>

In [30]:
for i in range(len(clean_answers)):
    clean_answers[i] += " <EOS>"

In [31]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [32]:
questions_into_int

[[764,
  1678,
  5535,
  6531,
  5226,
  8824,
  8824,
  6107,
  6671,
  8824,
  1367,
  285,
  6376,
  5253,
  8824,
  6031,
  8747,
  2122,
  4197,
  1876,
  8824,
  4809],
 [1730,
  8241,
  251,
  1678,
  5786,
  5662,
  5090,
  8824,
  7238,
  6760,
  12,
  844,
  5090,
  3646],
 [5844, 1876, 8824, 6107, 8824, 6107, 8824, 4228, 3027],
 [3646,
  1367,
  3431,
  8621,
  3042,
  6760,
  12,
  6392,
  5381,
  5082,
  12,
  6001,
  6318,
  4809],
 [2782, 2782, 6494, 12, 4773, 6377, 1678, 3405, 5844, 3266, 5378, 7509, 8824],
 [8752],
 [1876,
  3669,
  12,
  8752,
  8241,
  6816,
  832,
  1876,
  1881,
  1317,
  5378,
  8816,
  8824,
  5176,
  1317,
  1561,
  4773,
  4639,
  8241,
  764,
  5844,
  6775,
  2370,
  1418,
  8007],
 [3841],
 [8824,
  1460,
  1418,
  2917,
  3666,
  4070,
  1416,
  7171,
  8023,
  1418,
  531,
  4678,
  5875,
  162,
  6494,
  2162,
  8216,
  1902,
  1418,
  4400,
  8102,
  1317,
  6494,
  8579,
  3144],
 [5828, 7238, 2496, 1678, 3611, 6086, 2238, 5378, 5617],


In [33]:
answers_into_int

[[1730,
  8241,
  251,
  1678,
  5786,
  5662,
  5090,
  8824,
  7238,
  6760,
  12,
  844,
  5090,
  3646,
  8823],
 [5844, 1876, 8824, 6107, 8824, 6107, 8824, 4228, 3027, 8823],
 [844, 162, 7744, 6575, 1678, 5043, 3042, 6965, 5284, 8824, 8606, 5150, 8823],
 [6310, 6494, 8823],
 [8752, 8823],
 [1876,
  3669,
  12,
  8752,
  8241,
  6816,
  832,
  1876,
  1881,
  1317,
  5378,
  8816,
  8824,
  5176,
  1317,
  1561,
  4773,
  4639,
  8241,
  764,
  5844,
  6775,
  2370,
  1418,
  8007,
  8823],
 [2600, 1902, 1418, 3611, 5970, 5378, 6775, 3000, 3648, 8823],
 [8824,
  1460,
  1418,
  2917,
  3666,
  4070,
  1416,
  7171,
  8023,
  1418,
  531,
  4678,
  5875,
  162,
  6494,
  2162,
  8216,
  1902,
  1418,
  4400,
  8102,
  1317,
  6494,
  8579,
  3144,
  8823],
 [6760, 12, 5378, 4336, 8823],
 [8324, 8621, 737, 5082, 8241, 764, 3405, 8823],
 [584, 737, 3646, 1367, 1372, 8799, 1876, 8824, 8823],
 [8241,
  479,
  1177,
  3666,
  4975,
  7744,
  3666,
  5107,
  6760,
  1042,
  8241,
  1177,


### Sorting both the QnA based on length

In [34]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25 + 1):
    for i in enumerate(questions_into_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [35]:
sorted_clean_questions

[[8752],
 [3841],
 [5924],
 [7599],
 [3067],
 [2782],
 [8674],
 [2782],
 [3781],
 [8404],
 [3349],
 [5082],
 [3841],
 [3067],
 [3841],
 [2030],
 [5918],
 [6385],
 [5548],
 [221],
 [5082],
 [6460],
 [3781],
 [844],
 [3841],
 [3781],
 [8824],
 [4735],
 [1365],
 [3781],
 [1695],
 [8824],
 [8824],
 [8601],
 [7744],
 [8824],
 [8824],
 [5082],
 [2782],
 [5082],
 [8824],
 [7599],
 [584],
 [5082],
 [3267],
 [8824],
 [1293],
 [2782],
 [2782],
 [4654],
 [8824],
 [8005],
 [2782],
 [8142],
 [8023],
 [221],
 [2782],
 [3921],
 [6422],
 [221],
 [8393],
 [8393],
 [8393],
 [8393],
 [221],
 [7195],
 [3120],
 [844],
 [1372],
 [8601],
 [5082],
 [234],
 [1021],
 [8824],
 [8824],
 [5243],
 [221],
 [4917],
 [5082],
 [5082],
 [221],
 [1260],
 [1260],
 [1260],
 [1260],
 [1260],
 [1260],
 [844],
 [8601],
 [1416],
 [8601],
 [8142],
 [7744],
 [4775],
 [1260],
 [1260],
 [1260],
 [8824],
 [1260],
 [1260],
 [1416],
 [906],
 [7459],
 [4560],
 [221],
 [6318],
 [221],
 [1503],
 [221],
 [7469],
 [221],
 [1730],
 [5924],

In [36]:
sorted_clean_answers

[[1876,
  3669,
  12,
  8752,
  8241,
  6816,
  832,
  1876,
  1881,
  1317,
  5378,
  8816,
  8824,
  5176,
  1317,
  1561,
  4773,
  4639,
  8241,
  764,
  5844,
  6775,
  2370,
  1418,
  8007,
  8823],
 [8824,
  1460,
  1418,
  2917,
  3666,
  4070,
  1416,
  7171,
  8023,
  1418,
  531,
  4678,
  5875,
  162,
  6494,
  2162,
  8216,
  1902,
  1418,
  4400,
  8102,
  1317,
  6494,
  8579,
  3144,
  8823],
 [1569, 8823],
 [6405, 1902, 758, 3047, 3042, 3120, 3037, 8823],
 [3646, 1272, 3508, 6531, 7183, 8823],
 [844, 3646, 1367, 3566, 8381, 3666, 1134, 7744, 3666, 5586, 8823],
 [8358, 7369, 8823],
 [3646, 4428, 8083, 6186, 1985, 6494, 8823],
 [462, 8823],
 [5786, 3646, 3310, 5225, 8621, 5378, 3574, 8752, 8823],
 [4773,
  8270,
  8436,
  8241,
  3266,
  4400,
  5378,
  2567,
  7245,
  832,
  297,
  1876,
  8824,
  615,
  5155,
  1525,
  8823],
 [3315, 8824, 8799, 5378, 8687, 8823],
 [2957, 2162, 1902, 5378, 8330, 2486, 8823],
 [2436,
  6760,
  8241,
  4365,
  8241,
  5786,
  2030,
  340

# 2. Building the Seq2Seq model

### Creating placeholders for input and target

In [37]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None,None], name = "input")
    targets = tf.placeholder(tf.int32, [None,None], name = "target")
    lr = tf.placeholder(tf.float32, name = "learning_rate")        
    keep_prob = tf.placeholder(tf.float32, name = "keep_prob")  
    return inputs, targets, lr, keep_prob

In [38]:
def preprocess_targets(target, word2int, batch_size):
    left_side = tf.fill([batch_size,1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets,[0,0],[batch_size,-1],[1,1])
    preprocessed_targets = tf.concat([left_side,right_side],1)
    return preprocessed_targets

### Creating the Encoder RNN layer

In [39]:
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    #create the lstm layer
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state

In [40]:
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                              training_decoder_function,
                                                                                                              decoder_embedded_input,
                                                                                                              sequence_length,
                                                                                                              scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

In [41]:
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
    return test_predictions

In [42]:
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions

In [43]:
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions

# 3. Training the SEQ2SEQ model

In [44]:
#Setting the hyperparameters
epochs = 100
batch_size = 64
rnn_size = 512
num_layers = 3
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.01
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5

In [45]:
tf.reset_default_graph()
session = tf.InteractiveSession()

In [46]:
inputs, targets, lr, keep_prob = model_inputs()

In [47]:
sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')

In [48]:
input_shape = tf.shape(inputs)

In [49]:
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)

In [50]:
# Setting up the Loss Error, the Optimizer and Gradient Clipping
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                 tf.ones([input_shape[0],sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)

In [51]:
# Padding the sequences with the <PAD> token
# Q: [Who are you <PAD> <PAD> <PAD>]
# A: [<SOS> I am a bot <EOS>]
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]

In [52]:
# Splitting the data into batches of questions and answers
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions)//batch_size):
        start_index = batch_index*batch_size
        questions_in_batch = questions[start_index:start_index + batch_size]
        answers_in_batch = answers[start_index:start_index + batch_size]  
        padded_questions_in_batch=np.array( apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))  
        yield padded_questions_in_batch, padded_answers_in_batch

In [53]:
# Splitting the questions and answers into training and validation sets
training_validation_split = int(len(sorted_clean_questions)*0.15)
# this gets us the split index
training_questions = sorted_clean_questions[training_validation_split:]
training_answers = sorted_clean_answers[training_validation_split:]
validation_questions = sorted_clean_questions[:training_validation_split]
validation_answers = sorted_clean_answers[:training_validation_split]

In [ ]:
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 1000
checkpoint = "chatbot_weights.ckpt" # For Windows users, replace this line of code by: checkpoint = "./chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch,
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                       epochs,
                                                                                                                                       batch_index,
                                                                                                                                       len(training_questions) // batch_size,
                                                                                                                                       total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                       int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_validation_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print("Sorry I do not speak better, I need to practice more.")
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
    if early_stopping_check == early_stopping_stop:
        print("My apologies, I cannot speak better anymore. This is the best I can do.")
        break
print("Game Over")

Epoch:   1/100, Batch:    0/2703, Training Loss Error:  0.091, Training Time on 100 Batches: 514 seconds


# 4. Testing the model

In [ ]:
checkpoint = "./chatbot_weights.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, checkpoint)

In [ ]:
def convert_string2int(question, word2int):
    question = clean_text(question)
    return [word2int.get(word, word2int['<OUT>']) for word in question.split()]

### Setting up the chat

In [ ]:
while(True):
    question = input("You: ")
    if question == 'Goodbye':
        break
    question = convert_string2int(question, questionswords2int)
    question = question + [questionswords2int['<PAD>']] * (25 - len(question))
    fake_batch = np.zeros((batch_size, 25))
    fake_batch[0] = question
    predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.5})[0]
    answer = ''
    for i in np.argmax(predicted_answer, 1):
        if answersints2word[i] == 'i':
            token = ' I'
        elif answersints2word[i] == '<EOS>':
            token = '.'
        elif answersints2word[i] == '<OUT>':
            token = 'out'
        else:
            token = ' ' + answersints2word[i]
        answer += token
        if token == '.':
            break
    print('ChatBot: ' + answer)